## Код преназначен для парсинга новостей с сайта администрации Республики Коми
## The code is intended for parsing news from the website of the administration of the Komi Republic
Дата изменения: 29.11.2022
<br>
Date of change: 29.11.2022

Автор: Шилова надежда
<br>
Author: Nadejda Shilova

Примечание:
можно было бы сделать непрерывную запись сразу в файлы, чтобы уменьшить использование памяти, но на сайте администрации Коми есть блокировщик. На данный момент есть sleep в парсере, но он не всегда спасает от блокировки, поэтому разделение парсера и обработки более целесообразно.
<br>
Note: it would be possible to make a continuous write immediately to files to reduce memory usage, but there is a blocker on the website of the Komi administration. At the moment, there is sleep in the parser, but it does not always save from blocking, so separating the parser and processing is more appropriate.

In [32]:
# Необходимые библиотеки
# Required libraries
from bs4 import BeautifulSoup
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.styles import Border, Side, Alignment, NamedStyle
import pandas as pd
from razdel import sentenize
import re
import requests
from time import sleep

In [33]:
def parsing_news(news_rus):
    """Функция для сбора новостей с сайта администрации.

    Параметры:
            news_rus: номер новости в ссылке на русском
    Результат выполнения:
            [list_rus, list_komi]: список, который содержит два списка новостей
            на русском и коми языках


    Function for collecting news from the administration's website.

    Parameters:
        news_rus: the news number in the link in Russian
    Execution result:
        [list_rus, list_komi]: the list contains two lists of news
        in Russian and Komi languages

    """
    # Ссылка на новость на русском
    # Link to the news in Russian
    url_rus = "https://rkomi.ru/news/" + str(news_rus)
    
    # Получение страницы на русском
    # Getting the page in Russian
    request_rus = session.get(url_rus)
    soup_rus = BeautifulSoup(request_rus.text)
    
    list_rus = []
    
    # Получение заголовка на русском
    # Getting the title in Russian
    for tag in soup_rus.find_all("title"):
        title = re.search(r'[^|]*', tag.text)
        list_rus.append(title.group(0))
    
    # Получение текста новости на русском
    # Getting the news text in Russian
    for tag in soup_rus.find_all("p"):
        rus_text = tag.text
        rus_clean_text = rus_text.replace('\r\n', '').replace('\n', '')
        list_rus.append(rus_clean_text)
        
    
    # Редирект на коми язык
    # Redirect to Komi
    url_komi = "https://rkomi.ru/language/kv"
    
    # Получение страницы на коми
    # Getting the page om Komi
    request_komi = session.get(url_komi)
    soup_komi = BeautifulSoup(request_komi.text, 'lxml')
    
    list_komi = []
    
    # Получение заголовка на коми
    # Getting the title in Komi
    for tag in soup_komi.find_all("title"):
        title = re.search(r'[^|]*', tag.text)
        list_komi.append(title.group(0))
    
    # Получение текста новости на коми
    # Getting the news text in Komi
    for tag in soup_komi.find_all("p"):
        komi_text = tag.text
        komi_clean_text = komi_text.replace('\r\n', '').replace('\n', '')
        list_komi.append(komi_clean_text)
    
    # Редирект на русский язык
    # Redirect to Russian
    url_rus_re = "https://rkomi.ru/language/ru"
    request_rus = session.get(url_rus_re)
    
    # Перерыв секунда, перестраховка, чтобы не заблокировали
    # Break a second
    sleep(1)
    
    return [list_rus, list_komi]

In [34]:
# Запуск ссесии с браузером
# Starting a session with the browser
session = requests.Session() 
session.headers.update({
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.9; rv:45.0) Gecko/20100101 Firefox/45.0'
})

In [35]:
# 1074
start_ind_news = 1074  # Номер самой ранней новости для сбора данных The number of the earliest news to collect data
end_ind_news = 673  # Номер самой поздней новости для сбора данных The oldest news number for data collection

In [36]:
# Получение всех заданных новостей с сайта
# Getting all the specified news from the site
news_list = []
while start_ind_news >= end_ind_news:
    news_list.append(parsing_news(start_ind_news))
    start_ind_news -= 1

In [37]:
def split_sentence(news):
    """Функция для разделения данных на предложения.

    Параметры:
            news: новость в виде списка, включающая список на русском и список на коми языках
    Результат выполнения:
            [list_sentenize_komi, list_sentenize_rus]: список, который содержит два списка,
            разделённых на предложения новостей, на коми и русском языках


    Function for dividing data into sentences.
    
    Parameters:
            news: news in the form of a list, including a list in Russian and a list in Komi
    Execution result:
            [list_sentenize_komi, list_sentenize_rus]: a list that contains
            two lists divided into sentences news, in Komi and Russian

    """
    list_sentenize_rus = []
    list_sentenize_komi = []
    for ind_lang, language_news in enumerate(news):
        # Новость на русском
        # News in Russian
        if ind_lang == 0:
            for ind, sentence in enumerate(language_news):
                # Заголовок
                # Title
                if ind == 0:
                    list_sentenize_rus.append(sentence)
                # Тело новости
                # News content
                elif sentence not in ('', '\n'):
                    # Разделение на предложения
                    # Division into sentences
                    text_generator_rus = sentenize(sentence)
                    for text_sentenize in text_generator_rus:
                        list_sentenize_rus.append(text_sentenize.text)
                    del text_generator_rus
        # Новость на коми
        # News in Komi
        else:
            for ind, sentence in enumerate(language_news):
                # Заголовок
                # Title
                if ind == 0:
                    list_sentenize_komi.append(sentence)
                # Тело новости
                # News content
                elif sentence not in ('', '\n'):
                    # Разделение на предложения
                    # Division into sentences
                    text_generator_komi = sentenize(sentence)
                    for text_one_sentenize in text_generator_komi:
                        list_sentenize_komi.append(text_one_sentenize.text)
                    del text_generator_komi
    return [list_sentenize_komi, list_sentenize_rus]

In [79]:
# Создание файла excel
# Creating an excel
wb = Workbook()
ws = wb.active
# Присвоение имени листа
# Assigning a sheet name
ws.title = "Коми-рус-англ"
# Добавление заголовков таблицы
# Adding Table Headers
ws['A1'] = 'Коми язык'
ws['B1'] = 'Русский язык'

# Первая строчка заполнения данных
# First row for the data
start_row = 2

# Создание стиля ячейки
# Creating a cell style
style_table = NamedStyle(name="style_table")
style_table.alignment = Alignment(wrap_text=True)  # Выравнивание Alignment
thins = Side(border_style="thin", color="0000ff")  # Тип линии Type of line
style_table.border = Border(right=thins, left=thins, top=thins, bottom=thins)  # Границы таблицы Borders of table
# Ширина столбцов
# Column width
ws.column_dimensions['A'].width = 80
ws.column_dimensions['B'].width = 80

In [80]:
# Шаблон на все виды слова "который"
# Template for all kinds of the word "который"
regex_kotor = re.compile(r', котор(?=ыми|ый|ого|ому|ом|ая|ой|ую|ое|ого|ые|ых|ым)')
regex_split = re.compile(r'(?<=\.) (?=Котор)')

In [81]:
number_check = 1  # Номер новости для доработки News number for correction
for news in news_list:
    # Если новость существует на двух языках
    # If the news exists in two languages
    if news[0] != news[1]:
        processed_text = split_sentence(news)
        # Если количество предложений на двух языках совпадает
        # If the count of sentences in two language is the same
        if len(processed_text[0]) == len(processed_text[1]):
            # Добавление новости в общий excel
            # Adding news to excel
            for ind, text in enumerate(processed_text[0]):
                ws.cell(row=start_row, column=1, value=text).style = style_table
                ws.cell(
                    row=start_row,
                    column=2,
                    value=processed_text[1][ind]
                ).style = style_table
                start_row += 1
        else:
            # Создание датафрейма для файлов доработки,
            # так как простая запись в excel обрезает некоторые данные (причина не известна)
            # Датафрейм создаётся сразу с коми языком, русский обрабатывается дополнительно
            # Creating a dataframe for revision files,
            # since a simple entry in excel cuts off some data (the reason is not known)
            # The dataframe is created immediately with the Komi language, Russian is processed additionally
            df_check = pd.DataFrame({'Коми язык': processed_text[0]})
            df_check['Русский язык'] = ''
            # Работа со сложноподчинённом предложением "который"
            # Working with a complex sentence "который"
            ind = 0
            for text in processed_text[1]:
                if re.findall(regex_kotor, text):
                    new_text = text = regex_kotor.sub('. Котор', text)
                    split_new_text = re.split(regex_split, new_text)
                    for text_split in split_new_text:
                        try:
                            df_check.loc[[ind], 'Русский язык'] = text_split
                            ind += 1
                        except KeyError:
                            new_row = {'Коми язык': '', 'Русский язык': text_split}
                            df_check = pd.concat(
                                [df_check, pd.DataFrame(new_row, index=[0])],
                                axis=0,
                                ignore_index=True
                            )
                else:
                    try:
                        df_check.loc[[ind], 'Русский язык'] = text
                        ind += 1
                    except KeyError:
                        new_row = {'Коми язык': '', 'Русский язык': text}
                        df_check = pd.concat(
                            [df_check, pd.DataFrame(new_row, index=[0])],
                            axis=0,
                            ignore_index=True
                        )
            name_file = str(number_check) + '.xlsx'
            df_check.to_excel(
                name_file,
                sheet_name='Коми-рус',
                index=False
            )
            number_check += 1
# Сохранение обработанного файла
# Saving the processed file
wb.save('parsing_administration.xlsx')

In [84]:
# Добавления стиля на файлы для обработки
# Отдельно, так как lood_woorkbook закрывает все архивные файлы
# Adding a style to files for processing
# Separately, since lood_workbook closes all archive files
for number_file in range(1, number_check):
    name_file = str(number_file) + '.xlsx'
    wb_check = load_workbook(name_file)
    ws_check = wb_check['Коми-рус']
    ws_check.column_dimensions['A'].width = 40
    ws_check.column_dimensions['B'].width = 40
    for ind_column in range(1, ws_check.max_column+1):
        for ind_row in range(1, ws_check.max_row+1):
            ws_check.cell(row=ind_row, column=ind_column).style = style_table
    wb_check.save(name_file)

# Предыдущий вариант записи файлов для дополнительной обработки
Не используется, так как excel обрезает некоторые предложения по не понятным причинам.<br>
Предположительно, в данных есть ещё какой-то скрытый символ, который не получилось отловить.

In [53]:
number_check = 1  # Номер новости для доработки News number for correction
for news in news_list:
    # Если новость существует на двух языках
    # If the news exists in two languages
    if news[0] != news[1]:
        processed_text = split_sentence(news)
        # Если количество предложений на двух языках совпадает
        # If the count of sentences in two language is the same
        if len(processed_text[0]) == len(processed_text[1]):
            # Добавление новости в общий excel
            # Adding news to excel
            for ind, text in enumerate(processed_text[0]):
                ws.cell(row=start_row, column=1, value=text).style = style_table
                ws.cell(
                    row=start_row,
                    column=2,
                    value=processed_text[1][ind]
                ).style = style_table
                start_row += 1
        else:
            # Создание файла excel для ручного анализа
            # Creating an excel for manual analysis
            wb_check = Workbook()
            ws_check = wb_check.active
            # Присвоение имени листа
            # Assigning a sheet name
            ws_check.title = "Коми-руc"
            # Добавление заголовков таблицы
            # Adding Table Headers
            ws_check['A1'] = 'Коми язык'
            ws_check['B1'] = 'Русский язык'
            # Ширина столбцов
            # Column width
            ws_check.column_dimensions['A'].width = 40
            ws_check.column_dimensions['B'].width = 40
            # Заполенение excel коми предложениями
            # Filling out excel with Komi sentences
            start_row_check = 2
            for text in processed_text[0]:
                ws_check.cell(row=start_row_check, column=1, value=text).style = style_table
                start_row_check += 1
            # Заполенение excel русскими предложениями
            # Filling out excel with Russian sentences
            start_row_check = 2
            for text in processed_text[1]:
                # Работа со сложноподчинённом предложением "который"
                # Working with a complex sentence "который"
                if re.findall(regex_kotor, text):
                    new_text = text = regex_kotor.sub('. Котор', text)
                    split_new_text = re.split(regex_split, new_text)
                    for text_split in split_new_text:
                        ws_check.cell(row=start_row_check, column=2, value=text_split).style = style_table
                        start_row_check += 1
                else:
                    ws_check.cell(row=start_row_check, column=2, value=text).style = style_table
                    start_row_check += 1
            # Сохранение файла для ручного анализа
            # Save the file for manual analysis
            wb_check.save(str(number_check) + '.xlsx')
            number_check += 1
# Сохранение обработанного файла
# Saving the processed file
wb.save('parsing_administration.xlsx')